### Anteckningar / Erfarneheter från implementation

### ```if name == "__main__":```
- To run your script only when executed directly (not imported), use:  
```
if __name__ == "__main__":
    run() # eller main(), eller vilken annan funktion som man vill ska köras.
```
- Att använda denna innebär att man kommer köra koden (i run-funktionen) om man trycker på play alternativt om man startar filen genom terminalen. Men man kommer inte automatiskt köra koden om man importerar filen från en annan fil eller dylikt.
- Dvs, Koden som står inne i run()-funktionen kommer inte att köras vid import!

### Type hinting
#### i funktions definition:
Type hinting innebär att man i koden anger vilken datatyp som förväntas som output från en funktion:<br>
```
def calc_cost(data) -> int:
    cost = ....
```
#### i dataclass definition:
Inside a dataclass, typehint is also used:<br>
<br>
```
@dataclass
class Exercise:
    name: str
    reps: int
    sets: int
    weight: any
```
- any => accepterar alla datatyper
- Improves readability
- Does not enforce return type at runtime, vilket även gäller för dataclasses
- Det finns "typechecker" som Mypy

### Linting
- tbd

### init.py and packages:
- What is __init__.py?
- It marks a folder as a Python package.
- Without it (in older Python), you couldn’t import from that folder.
- Today (Python 3.3+), implicit namespace packages exist, but having __init__.py is still good practice because:
- You can control what’s imported when someone does ```from mypackage import *```.
- You can define package-level variables, logging setup, or helper functions.
- You can provide a package API surface — the “public entrypoint” for that folder.

## Flask
- [Geek for Gekks, Flask Tutorial](https://www.geeksforgeeks.org/python/flask-tutorial/)
- [Geek for Geeks, creating Rest API's with Flask](https://www.geeksforgeeks.org/python/flask-creating-rest-apis/)

### Lessons Learned från att jobbat med Flask API för household förbrukning
#### Flask
- En Flask app är en web server som väntar på HTTP requests. När en request kommer in, så matchar Flask den till en "route handler", som i sin tur kickar igång pipelinen.
- Ett API kan ta emot olika typer av requests. De flesta requests följer CRUD, dvs Create, Read, Update, Delete. Till API'et skickas ett API-request. ['GET'] motsvarar ungeför Read. ['POST'] innebär att skicka in data till API'et, som sedan ska användas för att tex beräkna något. ['PUT'] = Update, ['DELETE'] = Delete. Dessa är HTTP-metoder!
- Använder Flask som är ett lightweight webframework (micro-framework). Flask är byggt ovanpå två kraftfulla libraries: Werkzeug som är en WSGI server och Jinja som gör att man kan anväda dynamisk HTML.
- Flask använder sk **routes**, vilka mappar URL's till Python funktioner.
- Flask har stöd för RESTful API's och är populärt för att bygga API'er.
- Flask har en inbyggd utvecklingsserver, vilket gör det enkelt att utveckla och testa API'er lokalt.
- Man börjar med att skapa ett Flask **app-objekt**, vilket är den centrala Flask application instansen. Därefter vill man till slut köra run på detta objektet. Det är det som startar servern, som sedan ligger och väntar på att få API-requests till sig. Vid ett API-request så startar respektive "route".
- En bra regel enligt MG är att API'et alltid ska svara med ett svar, även om det inte är ett 200 svar
#### Why app.parser och app.predictor?<br>
ChatGPT föreslog att addera både parser-klassen och prediktor klassen som till app-objektet. Det verkar vara ett "common trick to stash dependencies in the app-object." Flask objektet som heter "app" är ett normalt Python objekt, till vilket man kan addera "arbitrary attributes".
```
app = Flask(__name__) #skapar ett app-objekt i Flask (eller en instans av API'et)<br>
#Nedan kod "attaches arbitrary attributes to the Flask object"
app.parser = Parser()
app.predictor = PredictionService()
register_routes(app) # innan man kör app.run så ska man definera routes'arna

app.run() # startar web servern
```
Routes: sk routes beskriver mappningen mellan URL's och den kod som ska köras:
```
@app.route("/health")
def return_health()
    return "status ok"
```
- För att prova sitt API lokalt kan man använda Postman eller som jag gjorde Thunder Client. Men den kan man skicka in olika API requests och se om man får tillbaka det förväntade svaret.
- Som ett av de första stegen i "routingen" är att accessa inkommande request data vid 'POST' kommnando, kan man använda **request**-objektet  
```
data = request.get_json() # omvandlar datan som bifogas till en json
req = RequestObject(**data) # omvandlar json datan till ett RequestObject (i vårt fall en DataClass)
```



### Dataclasses:
[Datacamp dataclass intro](https://www.datacamp.com/tutorial/python-data-classes?utm_cid=19589720821&utm_aid=157156375191&utm_campaign=230119_1-ps-other~dsa~tofu_2-b2c_3-emea_4-prc_5-na_6-na_7-le_8-pdsh-go_9-nb-e_10-na_11-na&utm_loc=9062342-&utm_mtd=-c&utm_kw=&utm_source=google&utm_medium=paid_search&utm_content=ps-other~emea-en~dsa~tofu~tutorial-python&gad_source=1&gad_campaignid=19589720821&gbraid=0AAAAADQ9WsEJN3V3FWNdQh9kQPiS_1OJ-&gclid=CjwKCAjw_fnFBhB0EiwAH_MfZpqxCq9uMTqK6aOQPTMyXgUgYsql7S5dQs5dTr0sxdvtGBWY6rzLCRoCmesQAvD_BwE)
- En ny sak för mig är det som kallas dataclasses.
- Dataclasses är i grunden vanliga klasser, men som kräver mycket mindre kod för att implementera samma funktionalitet.
- repr och eq metoderna redan implementerade
- repr = objekt representation, som innehåller allt för att återskapa objektet
- eq = equality operator, kan jämföra två objekt med varandra
- Det är ett sätt att "wrappa datan i en json fil". Fördelar:
- I dataclasses så anger man förväntade **datatyper** som type hints (but, dataclasses don't enforce types at runtime!). Dvs, man kan köra in andra datatyper än det som man definierat (tyvärr?). 
- man kan addera defaultvärden
- Dataclass'en innehåller precis allt det som API-anropet ska innehålla, dvs en specifikation hur API-anropet ska se ut
- Pydantic kan automat-generera dokumentation (dvs, användande av dataclasser ger möjlighet till integration med andra libraries)
- ```def post_init(self)```-metoden: om det finns en sådan metod definierad, så kommer den köras efter init (därav namnet post_init). Tex kan man köra tester på datan i dataclassen.
```
@dataclass
class Person:
    name: str
    phone: int

def __post_init__(self):
    if not....
    raise error...
```

### Felhantering (syntax errors and exceptions):
#### två typer av "errors":
    - "syntax errors": innebär att koden är fel i sin syntax (kallas också "parsing errors")
    - "exceptions": koden är syntaktiskt korrekt, men den orsakar ändå ett fel. Fel som hittas under execution kallas "exceptions". Tex: ZeroDivisionError, NameError, TypeError, ValueError
    - Det finns "built-in exceptions" och man kan skapa "user defined exceptions".
#### try - except
- Felhantering är hur programmet ska bete sig när något går fel i runtime
- Målet är att undvika att programmet krashar, och hantera felen snyggt.
- ```try / except osv```
#### raise
Nedan: utan felhantering krachar programmet, med felhantering returneras ett 400 Bad Request med ett tydligt meddelande.
```
def to_series(data):
    try:
        s = pd.Series(data["historical_data"])
    except KeyError:
        raise ValueError("Missing 'historical_data' in input")
    return s
```
Man skriver sin felhantering först, sedan skriver man sina tester. Detta eftersom testerna ska testa att felhanteringen fungerar.

### Koder
- 200: ok
- 400: bad request
- 404: not found
- 500: internal server error

### Testning (Pytest tex):
- Tester gör man för att testa koden innan deployment, för att se att den fungerar som den ska.
- Man skriver tester som "ska fungera".
- Man skriver också tester som testar att felhanteringen fungerar, tex skickar in tom information eller liknande.
- Målet är att hitta buggar och bevisa att programmet gör rätt saker.
- Tex Pytest, unittest etc
- Test Coverage

### GitHub Actions
- VSCode extension finns


### Logging

### Typing

### Argparser

### Enum

### Execution order
Frågade ChatGPT vad som händer när jag kör min main.py. Jag är specifikt intresserad av att förstå vilka funktioner som körs, och varför tex vi lade viss kod direkt i __init__.py-filen i varje folder.

When main.py is executed, Python processes all the import statements at the top of the file before running any other code. Here’s what happens for each import:

1. ```from api import create_app```<br>
Python looks for a module named api in the current directory or in the Python path.
In your project, api is likely a folder or file inside application (since you have sys.path.append(os.path.abspath('..')), the parent directory is added to the path).
Python executes the top-level code in api/__init__.py (if it exists) and then in api.py or api/__init__.py (depending on your structure).
Any code at the top level of these files (outside of functions/classes) will be executed immediately.
The create_app function is made available in main.py.
2. ```from application.my_parser.series_parser import Parser```<br>
```__init__.py``` i foldern exekeveras först, därefter executes the series_parser.py file inside my_parser.
Any **top-level code** in series_parser.py is executed.
The Parser class is made available in main.py.
3. ```from application.my_predictor.prediction_service import PredictionService```
```__init__.py``` i foldern exekeveras först,Python finds and executes the prediction_service.py file inside my_predictor.
Any top-level code in prediction_service.py is executed.
The PredictionService class is made available in main.py.<br>
  
#### Summary:<br>
For each import, Python runs all code at the top level of the imported module (not inside functions/classes). This can include variable assignments, function/class definitions, and any statements not inside a function or class. Only after all imports are processed does Python continue with the rest of main.py.  <br>
<br>
OBS, viktigt: För varje import så kör Python all kod i modulen på top level (tex klasser och funktioner identifieras och görs tillgängliga, men körs ej)<br>
<br>
Därefter cachas modulen i sys.modules, och den kommer inte köras igen om den importeras igen.<br>
<br>
##### Mer om ```__init__.py```:
När man importerar en modul så exekveras __init__.py filen alltid. You can use __init__.py to:  
- Expose selected functions/classes (from .parser import Parser)
- Set up package-level variables
- Or leave it empty (just to mark the folder as a package).
#### Tip:<br>
If you want to see exactly what runs, look for code in those files that is not indented (i.e., not inside a function or class). That code will execute on import.<br>
OBS!! Code inside ```if __name__ == "__main__":``` blocks will NOT run on import—only when that file is executed directly.

🔹 TL;DR

- Execution starts in the file you run directly.
- Imports execute the imported file once, then cache it.
- ```__init__.py``` runs whenever its package is first imported.
- You normally organize code into packages of related modules, import what you need, and keep a single main.py or app.py as entry point. (det är detta som magnus också säger till mig!)
- Man delar upp koden i paket för att undvika att det blir messy, enklare testa varje paket för sig utan att behöva starta Flask, kunna göra ändringar eller byta ut tex "prediktor" utan att behöva ändra någon annan stanns. Och det håller koden organiserad allteftersom projektet växer.
- När man gör egna moduler för Parser och Prediktor, som aldrig importerar Flask, så gör man den **"framework agnostic"**, dvs verkar som att man kan byta ut Flask mot ett annat framework, utan att ändra i Parser och Prediktor.

### Why put the "create app" function inside the ```__init__.py``` file?
Placing the Flask app creation code in the __init__.py file of the api folder has several benefits:<br>

#### Package Initialization:<br>
The __init__.py file is automatically executed when the package (api) is imported. This makes it a natural place to set up and configure the Flask app, ensuring it is ready whenever the package is used.
<br>
#### Single Entry Point:<br>
By centralizing app creation in __init__.py, you provide a single, consistent entry point for creating the Flask app. This makes it easier to import and use the app in different contexts (development, testing, production).
<br>
#### Cleaner Imports:<br>
Other modules can simply do ```from api import create_app``` (or ```from api import app```), without needing to know the internal structure of the package.
<br>
#### Encapsulation:<br>
All setup, configuration, and route registration for the API are encapsulated within the package, keeping the application modular and organized.
<br>
#### Testing and Reusability:<br>
Having the app creation logic in __init__.py makes it easier to create multiple app instances for testing or different configurations, as you can expose a create_app() factory function.
<br>
#### Summary:<br>
Allocating Flask app creation in __init__.py improves modularity, encapsulation, and usability of your API package, making your project easier to maintain and extend.

### Execptions, some coding examples

In [18]:
try:
    a = 3 + (3/0)
    print(f"a={a})")
except ZeroDivisionError:
    print('This is a ZeroDivisionError')

This is a ZeroDivisionError


#### "unhandled exception"
- means that the execution of a program stops with an error message

In [ ]:
# nedan kod är felaktig, då det inte är en ValueError som kastas
# utan en ZeroDivisionError
# kallas "unhandled exception"
try:
    a = 3 + (3/0)
    print(f"a={a})")
except ValueError:
    print('This is a ValueError')

ZeroDivisionError: division by zero

Ett try-statement kan ha multipla excpet-clauses för att hantera olika typer av exceptions. Bara en "handler" kommer exekveras.

In [22]:
try:
    a = 3 + (3/0)
    print(f"a={a})")
except ValueError:
    print('This is a ValueError')
except ZeroDivisionError:
    print('This is a ZeroDivisionError')


This is a ZeroDivisionError


Man kan addera detta:

In [23]:
try:
    a = 3 + (3/0)
    print(f"a={a})")
except ValueError:
    print('This is a ValueError')
except ZeroDivisionError as err:
    print('Error:', err)

Error: division by zero


Man kan ha flera typer av exceptions i en tuple:

In [20]:
try:
    a = 3 + (3/0)
    print(f"a={a})")
except ValueError:
    print('This is a ValueError')
except (ZeroDivisionError, ValueError, TypeError):
    print('This is a ZeroDivisionError, ValueError or TypeError')

This is a ZeroDivisionError, ValueError or TypeError


Exception handlers do not handle only exceptions that occur immediately in the try clause, but also those that occur inside functions that are called (even indirectly) in the try clause. For example:

In [24]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


#### Raising exceptions:
- The raise statement allows the programmer to force a specified exception to occur. 

In [28]:
def set(age):
    if age < 0:
        raise ValueError("Age cannot be negative.")
    print(f"Age set to {age}")

try:
    set("str")
except ValueError as e:
    print(e)
except TypeError as e:
    print(e)

'<' not supported between instances of 'str' and 'int'


### Custom exceptions
- You can also create custom exceptions by defining a new class that inherits from Python’s built-in Exception class. This is useful for application-specific errors. Let's see an example to understand how.

In [29]:
class AgeError(Exception):
    pass

def set(age):
    if age < 0:
        raise AgeError("Age cannot be negative.")
    print(f"Age set to {age}")

try:
    set(-5)
except AgeError as e:
    print(e)

Age cannot be negative.
